# Replicación del proceso de creación del dataset instruct-legal-refugiados, de somosnlp, disponible en hugginface https://huggingface.co/datasets/somosnlp/instruct-legal-refugiados-es

# Paso 1: Creacion de un dataset de textos legales sobre asilo, refugiados e immigración

In [1]:
!pip install datasets
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.7 MB/s eta 0

Leemos las credenciales de HF, primero, conecto a Google Drive i cambio el directorio de trabajo en el drive, donde tengo el fichero .env

In [2]:
import os
from google.colab import drive

drive.mount("/content/drive")

os.chdir('/content/drive/MyDrive/')



Mounted at /content/drive


In [3]:
!pip install python-dotenv


In [4]:
from dotenv import load_dotenv
import os

# Load the enviroment variables
load_dotenv()

True

In [5]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.schema import TransformComponent

import datasets

import re
from pprint import pprint

Leemos los archivos PDF y txt de la carpeta docschat con diferentes textos legales sobre el asilo politico, la protección internacional, procedimientos de aplicación a refugiados, principalmente en España, pero también en otros paises.

In [6]:
filename_fn = lambda filename: {"file_name": filename}

# automatically sets the metadata of each document according to filename_fn
documents = SimpleDirectoryReader(
    "./docschat", file_metadata=filename_fn
).load_data()

In [7]:
print(len(documents))

2002


In [8]:
print(len(documents[1000].text))
print(documents[1000].text)

3809
exista, el comercializador de último recurso con mayor cuota de mercado en la 
comunidad autónoma, deberá atender el suministro de aquellos consumidores que, sin 
tener derecho a acogerse a la tarifa de último recurso, transitoriamente carezcan de 
un contrato de suministro en vigor con un comercializador y continúen consumiendo 
gas.
Esta obligación se extiende únicamente durante el plazo de un mes desde la 
finalización del contrato del cliente.
Transcurrido dicho plazo sin que el consumidor disponga de un contrato en vigor 
de suministro con un comercializador, se procederá según lo previsto en el artículo 55 
del Real Decreto 1434/2002, de 27 de diciembre, por el que se regulan las actividades 
de transporte, distribución, comercialización, suministro y procedimientos de 
autorización de instalaciones de gas natural.
En el caso de que el consumidor se trate de un servicio esencial, de acuerdo con 
lo previsto en el artículo 60 del Real Decreto 1434/2002, de 27 de diciembre, la

Función básica para la limpieza de texto

In [9]:
class TextCleaner(TransformComponent):
    def __call__(self, nodes, **kwargs):
        for node in nodes:
            node.text = re.sub("\n\n+", ". ", node.text)
            node.text = re.sub("\n", " ", node.text)
            node.text = re.sub("\.+", ".", node.text)
            node.text = re.sub(" +", " ", node.text)
            #note.text = re.sub(r"[^ \w!\"$()=?¿¡!:,;+]", '', string)
        return nodes

Creamos el pipeline de limpieza y divisón de los textos, generando bloques de 512 tokens.

In [10]:
# use in a pipeline
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=512, chunk_overlap=0),
        TextCleaner(),
    ],
)

nodes = pipeline.run(documents=documents)

In [11]:
print(len(nodes))
pprint(nodes[100].get_content())
pprint(nodes[100].get_metadata_str())

4791
('39\xa0 \xa0interesa tratar varios aspectos que son pr esupuestos para su '
 'aplicación, esto es, la movilidad de los ciudadanos de la Unión y la de los '
 'nacionales de terceros Estados en la Unión. 3.1. El presupuesto: Ciudadanos '
 'de la Unión y Ciudadanos en la Unión. Los Tratados iniciales de las Comunida '
 'des se centraron en la libertad de circulación de los trabajadores, como '
 'factor necesario para la prosperidad del mercado La categoría de trabajador '
 'será supera da al integrarse en una más amplia: la ciudadanía de la Unión . '
 'No obstante, este prog reso inclusivo hacia la ciudadanía de la Unión, '
 'conforma, a su vez, otro grupo antagónico como es el de aquellos que no son '
 'nacionales de un Estado miembro y para los que – exclusivamente por esta '
 'razón - se legitima su exclusión. Será el Tratado de Ámsterdam, com '
 'unitarizando el asilo y la inmigración, el que dará un paso más al '
 'establecer mecanis mos de integración para los nacionales de te

Extraemos de los json nodes de Llamaindex los campos que necesitamos

In [12]:
import json

json_nodes=[]

for node in nodes:
    # Extraemos el nombre del fichero para
    source=node.metadata["file_name"].split("\\")[-1].split(".")[0]
    if "page_label" in node.metadata:
      page = node.metadata["page_label"]
    else:
      page=0

    data={
        "source": source,
        "page": str(page),
        "text_ini": node.start_char_idx,
        "text_fin": node.end_char_idx,
        "text": node.text,
    }
    json_nodes.append(data)


In [13]:
print(len(json_nodes))
json_nodes[100]

4791


{'source': '/content/drive/MyDrive/docschat/94_F06',
 'page': '40',
 'text_ini': 0,
 'text_fin': 1328,
 'text': '39\xa0 \xa0interesa tratar varios aspectos que son pr esupuestos para su aplicación, esto es, la movilidad de los ciudadanos de la Unión y la de los nacionales de terceros Estados en la Unión. 3.1. El presupuesto: Ciudadanos de la Unión y Ciudadanos en la Unión. Los Tratados iniciales de las Comunida des se centraron en la libertad de circulación de los trabajadores, como factor necesario para la prosperidad del mercado La categoría de trabajador será supera da al integrarse en una más amplia: la ciudadanía de la Unión . No obstante, este prog reso inclusivo hacia la ciudadanía de la Unión, conforma, a su vez, otro grupo antagónico como es el de aquellos que no son nacionales de un Estado miembro y para los que – exclusivamente por esta razón - se legitima su exclusión. Será el Tratado de Ámsterdam, com unitarizando el asilo y la inmigración, el que dará un paso más al estab

Creamos una función para generar un fichero jsonlines donde guardar los datos generados

In [14]:
def write_jsonlines(json_list, filename):
    with open(filename, 'w') as f:
        for json_dict in json_list:
            f.write(json.dumps(json_dict) + '\n')
    return None

In [15]:
write_jsonlines(json_nodes, "refugio-asilo-es.jsonl")

Leemos el jsonlines file

In [16]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="refugio-asilo-es.jsonl")


Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source', 'page', 'text_ini', 'text_fin', 'text'],
        num_rows: 4791
    })
})

In [18]:
dataset.push_to_hub("Saturdays/legal-refugiados-es")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Saturdays/legal-refugiados-es/commit/e21806aaf235e8f2059e24db6a3e5d9047dbc5a1', commit_message='Upload dataset', commit_description='', oid='e21806aaf235e8f2059e24db6a3e5d9047dbc5a1', pr_url=None, pr_revision=None, pr_num=None)

Recuperamos el dataset subido al HUF
#?????

In [19]:
prev_dataset = load_dataset("edumunozsala/legal-refugiados-es")

Generating train split:   0%|          | 0/5485 [00:00<?, ? examples/s]

In [20]:
prev_dataset, dataset

(DatasetDict({
     train: Dataset({
         features: ['source', 'page', 'text_ini', 'text_fin', 'text'],
         num_rows: 5485
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['source', 'page', 'text_ini', 'text_fin', 'text'],
         num_rows: 4791
     })
 }))

Ultimo registro del dataset antes de la concatenacion

In [21]:
pprint(prev_dataset['train'][-1])

{'page': '59',
 'source': 'reglamento_ue_604_2013_criterios_responsable_solicitud_proteccion_internacional',
 'text': 'DECLARACIÓN DEL CONSEJO, DEL PARLAMENTO EUROPEO Y DE LA COMISIÓN El '
         'Consejo y el Parlamento Europeo invitan a la Comisión a que, sin '
         'perjuicio de su derecho de iniciativa, estudie la conveniencia de '
         'revisar el artículo 8, apartado 4, de la refundición del Reglamento '
         'de Dublín, una vez que el Tribunal de Justicia dicte sentencia sobre '
         'el asunto C-648/11, MA, BT, DA/Secretary of State for the Home '
         'Department, y a más tardar, en los plazos fijados en el artículo 46 '
         'del Reglamento de Dublín. El Parlamento Europeo y el Consejo '
         'ejercerán después sus competencias legislativas, teniendo en cuenta '
         'el interés superior del menor. La Comisión, con ánimo conciliador y '
         'para garantizar la inmediata adopción de la propuesta, acepta '
         'examinar esta invitació

Concatenamos ambos dataset

In [22]:
new_dataset = datasets.concatenate_datasets([prev_dataset['train'],dataset['train']])
new_dataset

Dataset({
    features: ['source', 'page', 'text_ini', 'text_fin', 'text'],
    num_rows: 10276
})

Subimos el dataset concatenado a HF

In [23]:
new_dataset.push_to_hub("Saturdays/legal-refugiados-es")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/414 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Saturdays/legal-refugiados-es/commit/2678b336e47b5bf61bdce06ce385e9c84ad24460', commit_message='Upload dataset', commit_description='', oid='2678b336e47b5bf61bdce06ce385e9c84ad24460', pr_url=None, pr_revision=None, pr_num=None)

## 2ª Fase: Generación de un dataset sintético a partir de los datos recopilados utilizando la librería Distilabel de Argilla

Instalar Distilabel con las funcionalidades necesarias

In [1]:
%pip install -U distilabel[hf-inference-endpoints,argilla] -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
from distilabel.llms.huggingface.inference_endpoints import InferenceEndpointsLLM
from distilabel.steps.tasks.text_generation import TextGeneration
from distilabel.steps.tasks.self_instruct import SelfInstruct
from distilabel.pipeline import Pipeline

In [3]:
import distilabel
distilabel.__version__

'1.1.1'

# Introducción
En este tutorial se muestra como generar conjuntos de datos sintéticos en Español para entrenar y mejorar modelos del lenguaje en Español.

Para ello se utiliza distilabel de Argilla, una librería escalable para generar datasets para LLMs.

Este cuaderno provee una breve guía de introducción pero se recomienda leer la [documentación](https://distillabel.argilla.io/latest) y explorar opciones más avanzadas así como casos de uso interesantes, más allá del ejemplo utilizado aquí.

Generación de instrucciones con HF Inference endpoints
Con este apartado, los equipos pueden generar instrucciones en Español sobre distintos temas y para distintas aplicaciones. Aquí se muestra solo un ejemplo muy básico.

Para ejecutar este apartado es necesario formar parte de la organización SomosNLP en Hugging Face y configurar el token personal (nivel write) para poder hacer uso de la cuenta PRO.

Se ruega no sobrecargar la API de inferencia y hacer pruebas con pequeñas muestras hasta tener claro el caso de uso y en cualquier caso no generar datasets de más de 5000 ejemplos.

Comprobar acceso a Inference Endpoints

In [11]:
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

# change endpoint name and namespace once deployed
ENDPOINT_NAME = "mistralai/Mixtral-8x7B-Instruct-v0.1"
MODEL_ID0 = "meta-llama/Meta-Llama-3-8B-Instruct"
MODEL_ID1 = "mistralai/Mistral-7B-Instruct-v0.3"
MODEL_ID2 = "Qwen/Qwen2-7B-Instruct"

with Pipeline("pipe-name", description="My first pipe") as pipeline:

  llm0 = InferenceEndpointsLLM(
    model_id=MODEL_ID0,
    api_key=hf_token
  )

  llm1 = InferenceEndpointsLLM(
    model_id=MODEL_ID1,
    api_key=hf_token
  )

  llm2 = InferenceEndpointsLLM(
    model_id=MODEL_ID2,
    api_key=hf_token
  )


In [12]:
#lm.generate([{"input": "Generate a random joke in Spanish, just the joke, no greetings"}])
#llm.generate(inputs=[[{"role": "user", "content": "Generate a random joke in Spanish, just the joke, no greetings"},],],)
task0=TextGeneration(
        name="text_generation",
        llm=llm0
        )

task1=TextGeneration(
        name="text_generation",
        llm=llm1
        )

task2=TextGeneration(
        name="text_generation",
        llm=llm2
        )


In [13]:
task0.load()
next(task0.process(inputs=[{"instruction": "Generate a random joke in Spanish, just the joke, no greetings"}]))

/usr/local/lib/python3.10/dist-packages/distilabel/steps/tasks/base.py:162: UserWarning: `use_system_prompt` is set to `True`, but no `system_prompt` in input batch, so it will be ignored.
  return [self.format_input(input) for input in inputs]


[{'instruction': 'Generate a random joke in Spanish, just the joke, no greetings',
  'model_name': 'meta-llama/Meta-Llama-3-8B-Instruct',
  'generation': ' or farewells. ¡Buena suerte!\n\nHere is your random joke:\n\n¿Por qué la computadora se enfadó? ¡Porque tenía un virus!\n\n(Translation: Why did the computer get angry? Because it had a virus!)'}]

In [14]:
task1.load()
next(task1.process(inputs=[{"instruction": "Generate a random joke in Spanish, just the joke, no greetings"}]))

[{'instruction': 'Generate a random joke in Spanish, just the joke, no greetings',
  'model_name': 'mistralai/Mistral-7B-Instruct-v0.3',
  'generation': ' or explanations.\n\n"¿Por qué el perro no puede jugar al fútbol?\n\nPorque cuando se acerca al balón, se lo come."\n\nTranslation: "Why can\'t the dog play soccer? Because when it approaches the ball, it eats it."'}]

In [15]:
task2.load()
next(task2.process(inputs=[{"instruction": "Generate a random joke in Spanish, just the joke, no greetings"}]))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/distilabel/steps/tasks/base.py:65 in load                │
│                                                                                                  │
│    62 │   def load(self) -> None:                                                                │
│    63 │   │   """Loads the LLM via the `LLM.load()` method (done for safer serialization)."""    │
│    64 │   │   super().load()                                                                     │
│ ❱  65 │   │   self.llm.load()                                                                    │
│    66 │                                                                                          │
│    67 │   @abstractmethod                                                                        │
│    68 │   def format_output(                                                                     │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/distilabel/llms/huggingface/inference_endpoints.py:226   │
│ in load                                                                                          │
│                                                                                                  │
│   223 │   │   │   │   status.state not in {"Loadable", "Loaded"}                                 │
│   224 │   │   │   │   and status.framework != "text-generation-inference"                        │
│   225 │   │   │   ):                                                                             │
│ ❱ 226 │   │   │   │   raise ValueError(                                                          │
│   227 │   │   │   │   │   f"Model {self.model_id} is not currently deployed or is not running    │
│   228 │   │   │   │   )                                                                          │
│   229                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Model Qwen/Qwen2-7B-Instruct is not currently deployed or is not running the TGI framework

In [16]:
application_description = (
    "You are a AI legal assistant adept at answering a wide array of legal, reasoning, "
    "and general questions about immigrant or refugee citizens. Users of this assistant love to ask the assistant to think and outlines "
    "the solutions step by step. It expects complete questions from users providing all the details "
    "to solve the proposed problem or respond to legal questions. It covers general "
    "knowledge about legal, support, and immigration exercises, and real-life scenarios where legal and "
    "reasoning are important.  Highly important!! You can only generate text in SPANISH."
)

criteria_for_query_generation=(
    "Develop 5 user queries that can be received by the given AI application and applicable to the provided context. "
    "Emphasize diversity in verbs and linguistic structures within the model\'s textual capabilities.\n\n# Criteria "
    "for Queries\nIncorporate a diverse range of verbs, avoiding repetition.\nEnsure queries are compatible with AI "
    "model\'s text generation functions and are limited to 1-2 sentences.\nDesign queries to be self-contained and "
    "standalone.\nBlend interrogative (e.g., What is the significance of x?) and imperative (e.g., Detail the "
    "process of x.) styles.\nWrite each query on a separate line and avoid using numbered lists or bullet points.\n\n"

)

instruction_task = []
for llm in (llm0, llm1):
  instruction_task.append(
      SelfInstruct(
        llm=llm,
        #group_generations=True,
        add_raw_output=True,
        #num_generations=1,
        criteria_for_query_generation=criteria_for_query_generation,
        application_description=application_description,
        num_instructions=5,
      )
  )


In [17]:
#Cambiamos los métodos "input_args_names" y "output_args_names" por "inputs" y "outputs" respectivamente
print(f"`SelfInstructTask[0]`\n   - Input arguments: {instruction_task[0].inputs}\n   - Output arguments: {instruction_task[0].outputs}")

`SelfInstructTask[0]`
   - Input arguments: ['input']
   - Output arguments: ['instructions', 'model_name']


In [18]:
print(f"`SelfInstructTask[1]`\n   - Input arguments: {instruction_task[0].inputs}\n   - Output arguments: {instruction_task[0].outputs}")

`SelfInstructTask[1]`
   - Input arguments: ['input']
   - Output arguments: ['instructions', 'model_name']


In [19]:
from datasets import Dataset, load_dataset

dataset = load_dataset("Saturdays/legal-refugiados-es")
dataset = dataset.remove_columns(column_names=['source', 'page', 'text_ini', 'text_fin'])
dataset = dataset.rename_columns({"text": "input"})
dataset

Generating train split:   0%|          | 0/10276 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input'],
        num_rows: 10276
    })
})

In [20]:
minidata = dataset['train'].select(range(0,10))
minidata

Dataset({
    features: ['input'],
    num_rows: 10
})

In [21]:
instruction_task[0].load()
instruction_task[0].format_input(minidata[0])

[{'role': 'user',
  'content': "# Task Description\nDevelop 5 user queries that can be received by the given AI application and applicable to the provided context. Emphasize diversity in verbs and linguistic structures within the model's textual capabilities.\n\n# Criteria for Queries\nDevelop 5 user queries that can be received by the given AI application and applicable to the provided context. Emphasize diversity in verbs and linguistic structures within the model's textual capabilities.\n\n# Criteria for Queries\nIncorporate a diverse range of verbs, avoiding repetition.\nEnsure queries are compatible with AI model's text generation functions and are limited to 1-2 sentences.\nDesign queries to be self-contained and standalone.\nBlend interrogative (e.g., What is the significance of x?) and imperative (e.g., Detail the process of x.) styles.\nWrite each query on a separate line and avoid using numbered lists or bullet points.\n\n\nWrite each query on a separate line and avoid using 

In [22]:
instruction_task[1].load()
instruction_task[1].format_input(minidata[0])

[{'role': 'user',
  'content': "# Task Description\nDevelop 5 user queries that can be received by the given AI application and applicable to the provided context. Emphasize diversity in verbs and linguistic structures within the model's textual capabilities.\n\n# Criteria for Queries\nDevelop 5 user queries that can be received by the given AI application and applicable to the provided context. Emphasize diversity in verbs and linguistic structures within the model's textual capabilities.\n\n# Criteria for Queries\nIncorporate a diverse range of verbs, avoiding repetition.\nEnsure queries are compatible with AI model's text generation functions and are limited to 1-2 sentences.\nDesign queries to be self-contained and standalone.\nBlend interrogative (e.g., What is the significance of x?) and imperative (e.g., Detail the process of x.) styles.\nWrite each query on a separate line and avoid using numbered lists or bullet points.\n\n\nWrite each query on a separate line and avoid using 

In [23]:
instruction_task[0].load()

inputs=list(minidata)

distiset0=next(instruction_task[0].process(inputs))

In [24]:
distiset0[1]

{'input': 'Trata, asimismo, de los efectos de las resoluciones  favorables o desfavorables a las solicitudes de asilo, y los recursos que pueden interponerse  frente a una resolución desfavorable, tanto en vía administrativa como jurisdiccional,  regulando en especial el contenido y efectos del informe favorable del Alto Comisionado de  las Naciones Unidas para los Refugiados en el procedimiento de inadmisión a trámite en  fronteras. Se contempla, finalmente, la situación especial que plantean los desplazados a  consecuencia de conflictos o disturbios graves de carácter político, étnico o religioso, para  los que se crea una cobertura legal específica que incluye su acceso a las estructuras  asistenciales previstas para los solicitantes de asilo y refugiados. En su virtud, a propuesta de los Ministros de Asuntos Exteriores, de Justicia e Interior y  de Asuntos Sociales; con el informe favorable de la Comisión Interministerial de Extranjería;  con la aprobación del Ministro para las Adm

In [25]:
instruction_task[1].load()

inputs=list(minidata)

distiset1=next(instruction_task[1].process(inputs))

In [26]:
distiset1[1]

{'input': 'Trata, asimismo, de los efectos de las resoluciones  favorables o desfavorables a las solicitudes de asilo, y los recursos que pueden interponerse  frente a una resolución desfavorable, tanto en vía administrativa como jurisdiccional,  regulando en especial el contenido y efectos del informe favorable del Alto Comisionado de  las Naciones Unidas para los Refugiados en el procedimiento de inadmisión a trámite en  fronteras. Se contempla, finalmente, la situación especial que plantean los desplazados a  consecuencia de conflictos o disturbios graves de carácter político, étnico o religioso, para  los que se crea una cobertura legal específica que incluye su acceso a las estructuras  asistenciales previstas para los solicitantes de asilo y refugiados. En su virtud, a propuesta de los Ministros de Asuntos Exteriores, de Justicia e Interior y  de Asuntos Sociales; con el informe favorable de la Comisión Interministerial de Extranjería;  con la aprobación del Ministro para las Adm

In [27]:
import pandas as pd

df0 = pd.DataFrame(distiset0)
df1 = pd.DataFrame(distiset1)

dataset0 = Dataset.from_pandas(df0)
dataset1 = Dataset.from_pandas(df1)


In [40]:
dataset0
dataset1.features

{'input': Value(dtype='string', id=None),
 'model_name': Value(dtype='string', id=None),
 'instructions': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'distilabel_metadata': {'raw_output_self_instruct_0': Value(dtype='string', id=None)}}

In [31]:
dataset1.save_to_disk("./content/gdrive/MyDrive/refugiats")

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

In [43]:
dataset0.save_to_disk("content/gdrive/MyDrive/refugiats")

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

In [29]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [30]:
#!rm /content/gdrive/MyDrive/Projects/QA-Legal-SomosNLP/dataset/selfinstruct/*
!ls -l /content/gdrive/MyDrive/refugiats

total 4
drwx------ 2 root root 4096 May  7 15:23 docschat


In [46]:
!cp ./dataset0 /content/gdrive/MyDrive/refugiats/

cp: cannot stat './dataset0': No such file or directory


In [76]:
dataset0.push_to_hub("Saturdays/legal-refugiados-es-instruct-llama")
dataset1.push_to_hub("Saturdays/legal-refugiados-es-instruct-mistral")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Saturdays/legal-refugiados-es-instruct-mistral/commit/7a7212121bbdb0805814be6e316be5c144b36b0a', commit_message='Upload dataset', commit_description='', oid='7a7212121bbdb0805814be6e316be5c144b36b0a', pr_url=None, pr_revision=None, pr_num=None)

In [32]:
rows = []
from datasets import Dataset

generations = []
raw_inputs=[]
for row in dataset0:
    for instructions in row["instructions"]:
        for generation in instructions:
            generations.append(generation)
            raw_inputs.append(row["input"])

generation_dataset0 = Dataset.from_dict({"input": raw_inputs, "question": generations})


In [33]:
print(generation_dataset0)
generation_dataset0[-1]

Dataset({
    features: ['input', 'question'],
    num_rows: 5190
})


{'input': 'Los solicitantes de asilo que se encuentren en territorio nacional serán informados por  la autoridad a la que se dirijan de la necesidad de aportar las pruebas o indicios en que base  su solicitud, así como de los derechos que le corresponden de conformidad con la Ley  5/1984, reguladora del derecho de asilo y de la condición de refugiado, en particular. del  derecho a intérprete y a la asistencia letrada. Asimismo, la autoridad ante quien se hubiese BOLETÍN OFICIAL DEL ESTADO LEGISLACIÓN CONSOLIDADA Página 4',
 'question': 'f'}

In [34]:
rows = []
from datasets import Dataset

generations = []
raw_inputs=[]
for row in dataset1:
    for instructions in row["instructions"]:
        for generation in instructions:
            generations.append(generation)
            raw_inputs.append(row["input"])

generation_dataset1 = Dataset.from_dict({"input": raw_inputs, "question": generations})


In [35]:
print(generation_dataset1)
generation_dataset1[-1]

Dataset({
    features: ['input', 'question'],
    num_rows: 3722
})


{'input': 'Los solicitantes de asilo que se encuentren en territorio nacional serán informados por  la autoridad a la que se dirijan de la necesidad de aportar las pruebas o indicios en que base  su solicitud, así como de los derechos que le corresponden de conformidad con la Ley  5/1984, reguladora del derecho de asilo y de la condición de refugiado, en particular. del  derecho a intérprete y a la asistencia letrada. Asimismo, la autoridad ante quien se hubiese BOLETÍN OFICIAL DEL ESTADO LEGISLACIÓN CONSOLIDADA Página 4',
 'question': 'a'}

In [36]:
len(generation_dataset0)
generation_dataset1[-2]

{'input': 'Los solicitantes de asilo que se encuentren en territorio nacional serán informados por  la autoridad a la que se dirijan de la necesidad de aportar las pruebas o indicios en que base  su solicitud, así como de los derechos que le corresponden de conformidad con la Ley  5/1984, reguladora del derecho de asilo y de la condición de refugiado, en particular. del  derecho a intérprete y a la asistencia letrada. Asimismo, la autoridad ante quien se hubiese BOLETÍN OFICIAL DEL ESTADO LEGISLACIÓN CONSOLIDADA Página 4',
 'question': 'ñ'}

In [61]:
# REducimos el tamaño para pruebas, si procede
#sample_dataset = generation_dataset.select(range(max_size))
#print("Tamaño del dataset inicial: ",len(sample_dataset))
#Eliminamos dsuplicados
df_dataset = generation_dataset0.to_pandas().drop_duplicates()
print(len(df_dataset))
print(df_dataset.head())

414
                                               input question
0  Real Decreto 203/1995, de 10 de febrero, por e...        H
1  Real Decreto 203/1995, de 10 de febrero, por e...        e
2  Real Decreto 203/1995, de 10 de febrero, por e...        r
4  Real Decreto 203/1995, de 10 de febrero, por e...         
5  Real Decreto 203/1995, de 10 de febrero, por e...        a


In [19]:
import argilla as rg

argilla_api_key = "admin.apikey"
argilla_space_url = "https://enricarmengol-argilla-rerfugiados.hf.space"

workspace = "admin"

rg.init(
    api_key=argilla_api_key,
    api_url=argilla_space_url,
    workspace=workspace
)

In [48]:
df1.head()

,input,model_name,instructions,distilabel_metadata
0,"Real Decreto 203/1995, de 10 de febrero, por e...",mistralai/Mistral-7B-Instruct-v0.3,[1. ¿Cómo se determina la condición de refugia...,{'raw_output_self_instruct_0': '1. ¿Cómo se de...
1,"Trata, asimismo, de los efectos de las resoluc...",mistralai/Mistral-7B-Instruct-v0.3,[1. ¿Cómo se procede en caso de que una solici...,{'raw_output_self_instruct_0': '1. ¿Cómo se pr...
2,Artículo único. Se aprueba el Reglamento par...,mistralai/Mistral-7B-Instruct-v0.3,[1. ¿Cómo se estructura la Oficina de Asilo y ...,{'raw_output_self_instruct_0': '1. ¿Cómo se es...
3,El reconocimiento de la condición de refugiado...,mistralai/Mistral-7B-Instruct-v0.3,[1. ¿Cómo se define la Comisión Interministeri...,{'raw_output_self_instruct_0': '1. ¿Cómo se de...
4,funciones de Secretario de la Comisión Intermi...,mistralai/Mistral-7B-Instruct-v0.3,[1. ¿Cómo deben establecerse y revisar periódi...,{'raw_output_self_instruct_0': '1. ¿Cómo deben...


In [94]:
ds0 = rg.FeedbackDataset(
    fields=[
        rg.TextField(name="input"),
        rg.TextField(name="questions")
    ],
    questions =[
        rg.RatingQuestion(
            name="rating",
            title="Rate the quality of the response:",
            description="1 = very bad - 5= very good",
            required=True,
            values=[1,2,3,4,5]
        )]
)

ds1 = rg.FeedbackDataset(
    fields=[
        rg.TextField(name="input"),
        rg.TextField(name="questions")
    ],
    questions =[
        rg.RatingQuestion(
            name="rating",
            title="Rate the quality of the response:",
            description="1 = very bad - 5= very good",
            required=True,
            values=[1,2,3,4,5]
        )]
)

ds0.push_to_argilla(name="legal-refugiados-es-isntruct-llama", workspace="admin")
ds1.push_to_argilla(name="legal-refugiados-es-instruct-mistral", workspace="admin")

INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully pushed to Argilla
INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset(
   id=1cdd6540-d2f8-424d-b5f6-b7920c6ff03f
   name=legal-refugiados-es-isntruct-llama
   workspace=Workspace(id=cab6ae65-dda0-48f6-b1a5-4d030cc9fd17, name=admin, inserted_at=2024-06-11 18:00:41.565706, updated_at=2024-06-11 18:00:41.565706)
   url=https://enricarmengol-argilla-rerfugiados.hf.space/dataset/1cdd6540-d2f8-424d-b5f6-b7920c6ff03f/annotation-mode
   fields=[RemoteTextField(id=UUID('580df1a5-42ff-448b-92f7-20979869c438'), client=None, name='input', title='Input', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('6c0e29a3-0b5b-4619-9496-510e568faaa8'), client=None, name='questions', title='Questions', required=True, type='text', use_markdown=False)]
   questions=[RemoteRatingQuestion(id=UUID('9b67b76c-e5db-473f-a893-f36a5575e548'), client=None, name='rating', title='Rate the quality of the res

RemoteFeedbackDataset(
   id=ee1208a1-3193-437a-93ed-36ede03cbfee
   name=legal-refugiados-es-instruct-mistral
   workspace=Workspace(id=cab6ae65-dda0-48f6-b1a5-4d030cc9fd17, name=admin, inserted_at=2024-06-11 18:00:41.565706, updated_at=2024-06-11 18:00:41.565706)
   url=https://enricarmengol-argilla-rerfugiados.hf.space/dataset/ee1208a1-3193-437a-93ed-36ede03cbfee/annotation-mode
   fields=[RemoteTextField(id=UUID('859cd274-1de5-4b88-aa83-9a013ab62a46'), client=None, name='input', title='Input', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('060951c3-2b5f-4b56-b23d-809728618b77'), client=None, name='questions', title='Questions', required=True, type='text', use_markdown=False)]
   questions=[RemoteRatingQuestion(id=UUID('2c6ae437-5570-4e42-8fb8-049ff5ef580d'), client=None, name='rating', title='Rate the quality of the response:', description='1 = very bad - 5= very good', required=True, type='rating', values=[1, 2, 3, 4, 5])]
   guidelines=None
   metadata_

In [105]:


records_pandas0 = []
for i, item in enumerate(dataset0):
    ds0.add_records(
        rg.FeedbackRecord(
            fields={
                "input": item["input"],
                "instructions": item["instructions"],
            },
            external_id=f"record-{i}"
        )
    )

    records_pandas1 = []
for i, item in df1.iterrows():
    records_pandas1.append(
        rg.FeedbackRecord(
            fields={
                "input": item["input"],
                "instructions": item["instructions"],
            },
            external_id=f"record-{i}"
        )
    )

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:4                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pydantic/v1/main.py:341 in __init__                      │
│                                                                                                  │
│    338 │   │   # Uses something other than `self` the first arg to allow "self" as a settable a  │
│    339 │   │   values, fields_set, validation_error = validate_model(__pydantic_self__.__class_  │
│    340 │   │   if validation_error:                                                              │
│ ❱  341 │   │   │   raise validation_error                                                        │
│    342 │   │   try:                                                                              │
│    343 │   │   │   object_setattr(__pydantic_self__, '__dict__', values)                         │
│    344 │   │   except TypeError as e:                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValidationError: 1 validation error for FeedbackRecord
fields -> instructions
  str type expected (type=type_error.str)

In [77]:
rg_dataset0 = dataset0.to_argilla(vector_strategy=False, metric_strategy=False)
rg_dataset1 = dataset1.to_argilla(vector_strategy=False, metric_strategy=False)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'Dataset' object has no attribute 'to_argilla'

In [30]:
from datasets import Dataset, load_dataset

dataset = load_dataset("Saturdays/legal-refugiados-es")
dataset = dataset.remove_columns(column_names=['source', 'page', 'text_ini', 'text_fin'])
dataset = dataset.rename_columns({"text": "input"})
dataset

DatasetDict({
    train: Dataset({
        features: ['input'],
        num_rows: 10276
    })
})

In [31]:
dataset.push_to_hub("Saturdays/legal-refugiados-es-net")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Saturdays/legal-refugiados-es-net/commit/ed502f1e92be9c07acd30d029c286e98496e6da9', commit_message='Upload dataset', commit_description='', oid='ed502f1e92be9c07acd30d029c286e98496e6da9', pr_url=None, pr_revision=None, pr_num=None)

In [4]:
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

# change endpoint name and namespace once deployed
ENDPOINT_NAME = "mistralai/Mixtral-8x7B-Instruct-v0.1"
MODEL_ID0 = "meta-llama/Meta-Llama-3-8B-Instruct"
MODEL_ID1 = "mistralai/Mistral-7B-Instruct-v0.3"
MODEL_ID2 = "Qwen/Qwen2-7B-Instruct"

llm0 = InferenceEndpointsLLM(
    model_id=MODEL_ID0,
    api_key=hf_token
  )

llm1 = InferenceEndpointsLLM(
    model_id=MODEL_ID1,
    api_key=hf_token
  )

llm2 = InferenceEndpointsLLM(
    model_id=MODEL_ID2,
    api_key=hf_token
  )


application_description = (
    "You are a AI legal assistant adept at answering a wide array of legal, reasoning, "
    "and general questions about immigrant or refugee citizens. Users of this assistant love to ask the assistant to think and outlines "
    "the solutions step by step. It expects complete questions from users providing all the details "
    "to solve the proposed problem or respond to legal questions. It covers general "
    "knowledge about legal, support, and immigration exercises, and real-life scenarios where legal and "
    "reasoning are important.  Highly important!! You can only generate text in SPANISH."
)

criteria_for_query_generation=(
    "Develop 5 user queries that can be received by the given AI application and applicable to the provided context. "
    "Emphasize diversity in verbs and linguistic structures within the model\'s textual capabilities.\n\n# Criteria "
    "for Queries\nIncorporate a diverse range of verbs, avoiding repetition.\nEnsure queries are compatible with AI "
    "model\'s text generation functions and are limited to 1-2 sentences.\nDesign queries to be self-contained and "
    "standalone.\nBlend interrogative (e.g., What is the significance of x?) and imperative (e.g., Detail the "
    "process of x.) styles.\nWrite each query on a separate line and avoid using numbered lists or bullet points.\n\n"
)

In [10]:
#CODI PER ACLARIR EL FUNCIONAMENT DEL PIPELINE

from distilabel.steps import LoadHubDataset

with Pipeline("Dataset_generator", description="pipeline to generate a dataset") as pipeline:

 load_ds = LoadHubDataset(name="load_ds", repo_id="Saturdays/legal-refugiados-es-net")

 #instruction_task = []
 #for llm in (llm0, llm1):
  #instruction_task.append(
 instruction_task=SelfInstruct(
        name="instruction_task",
        llm=llm0,
        #group_generations=True,
        add_raw_output=True,
        #num_generations=1,
        criteria_for_query_generation=criteria_for_query_generation,
        application_description=application_description,
        num_instructions=5,
      )


 result=instruction_task.process(load_ds)

 load_ds >> instruction_task

if __name__=="__main__":

  distiset0 = pipeline.dry_run(
      parameters={
          load_ds.name: {
              #"repo_id": "Saturdays/legal-refugiados-es-net",
              "split": "train"
          },
      },
      batch_size=1,
  )

[06/17/24 08:58:56] INFO     ['distilabel.pipeline.local'] 🌵 Dry run mode                             ]8;id=573263;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=308459;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py#106\106]8;;\

                    INFO     ['distilabel.pipeline.local'] 📝 Pipeline data will be written to         ]8;id=764825;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=506832;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py#128\128]8;;\
                             '/root/.cache/distilabel/pipelines/Dataset_generator/095f8b09341e13f11976             
                             c3bc2bb550efd2d90b5c/data'                                                            

                    INFO     ['distilabel.pipeline.local'] ⏳ Waiting for all the steps to load...     ]8;id=737590;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=28127;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py#469\469]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 28>:30                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py:277 in dry_run               │
│                                                                                                  │
│    274 │   │   │   │   │   parameters = {}                                                       │
│    275 │   │   │   │   parameters[step_name] = {"batch_size": batch_size}                        │
│    276 │   │                                                                                     │
│ ❱  277 │   │   distiset = self.run(parameters=parameters, use_cache=False)                       │
│    278 │   │                                                                                     │
│    279 │   │   self._dry_run = False                                                             │
│    280 │   │   return distiset                                                                   │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py:150 in run                  │
│                                                                                                  │
│    147 │   │   │   │   write_buffer.close()                                                      │
│    148 │   │   │   │   self._batch_manager = None                                                │
│    149 │   │   │   │   stop_logging()                                                            │
│ ❱  150 │   │   │   │   raise RuntimeError(                                                       │
│    151 │   │   │   │   │   "Failed to load all the steps. Could not run pipeline."               │
│    152 │   │   │   │   ) from _SUBPROCESS_EXCEPTION                                              │
│    153                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Failed to load all the steps. Could not run pipeline.

#Generar dataset de preguntas o instrucciones por bloques de textos de los documentos
Leemos el dataset con los bloques de texto extraidos de los documentos recopilados sobre leyes y procedimientos referidos a asiliados y refugiados

Seleccionamos un subconjunto de los registros para poder realizar el proceso de manera iterativa y minimizar el riesgo de errores en la inferencia.

Te invitamos a probar diferentes prompts y ver cuál da mejores resultados:

#Creamos un SelfInstructTask con la descrición apropiada a nuestro caso de uso:

Definirmos la frecuencia de los checkpoints, que nos permitiran recuperar las inferencias realizadas hasta el momento de prodcirse un posible error.

In [ ]:
from pathlib import Path
from distilabel.dataset import DatasetCheckpoint

# Assuming we want to save the dataset every 10% of the records generated.

freq = len(minidata) // 6
dataset_checkpoint = DatasetCheckpoint(path=Path.cwd() / "checkpoint_inst_folder", save_frequency=freq)

ModuleNotFoundError: No module named 'distilabel'

## Reorganizar los campos y formatear el dataset de instructions generado con distillabel (1ª fase)

In [ ]:
#Si no tienes la libreria datsets instalada
!pip install datasets

In [ ]:
from datasets import load_dataset, Dataset
import re

In [ ]:
dataset = load_dataset("edumunozsala/prueba-selfins-legal-es", download_mode='force_redownload', ignore_verifications=True)
dataset


/usr/local/lib/python3.10/dist-packages/datasets/load.py:2555: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(


Generating train split:   0%|          | 0/11105 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'generation_model', 'generation_prompt', 'raw_generation_responses', 'generations'],
        num_rows: 11105
    })
})

Convertimos a Pandas

In [ ]:
pandas_df = dataset['train'].to_pandas()
pandas_df.head()

,input,generation_model,generation_prompt,raw_generation_responses,generations
0,Below is a question in Spanish paired with a c...,[mistralai/Mixtral-8x7B-Instruct-v0.1],[<s>[INST] <<SYS>>\nYou are a AI legal assista...,"[ La Ley 5/1984, relativa al derecho de asilo ...","[ La Ley 5/1984, relativa al derecho de asilo ..."
1,Below is a question in Spanish paired with a c...,[mistralai/Mixtral-8x7B-Instruct-v0.1],[<s>[INST] <<SYS>>\nYou are a AI legal assista...,[ El proceso de inadmisión a trámite estableci...,[ El proceso de inadmisión a trámite estableci...
2,Below is a question in Spanish paired with a c...,[mistralai/Mixtral-8x7B-Instruct-v0.1],[<s>[INST] <<SYS>>\nYou are a AI legal assista...,[ Los efectos de una resolución favorable a un...,[ Los efectos de una resolución favorable a un...
3,Below is a question in Spanish paired with a c...,[mistralai/Mixtral-8x7B-Instruct-v0.1],[<s>[INST] <<SYS>>\nYou are a AI legal assista...,[ El informe favorable del Alto Comisionado de...,[ El informe favorable del Alto Comisionado de...
4,Below is a question in Spanish paired with a c...,[mistralai/Mixtral-8x7B-Instruct-v0.1],[<s>[INST] <<SYS>>\nYou are a AI legal assista...,[ El artículo único del Reglamento para la apl...,[ El artículo único del Reglamento para la apl...


In [ ]:
from pprint import pprint

pprint(dataset['train'][-1])
pprint(dataset['train'][0]['input'])

{'generation_model': ['mistralai/Mixtral-8x7B-Instruct-v0.1'],
 'generation_prompt': ['<s>[INST] <<SYS>>\n'
                       'You are a AI legal assistant adept at answering a wide '
                       'array of legal, reasoning, and general questions about '
                       'immigrant or refugee citizens. Users of this assistant '
                       'love to ask the assistant to think and outlines the '
                       'solutions step by step. It expects complete questions '
                       'from users providing all the details to solve the '
                       'proposed problem or respond to legal questions. It '
                       'covers general knowledge about legal, support, and '
                       'immigration exercises, and real-life scenarios where '
                       'legal and reasoning are important.  Highly important!! '
                       'You can only generate text in SPANISH.<</SYS>>\n'
                       '\n'

# Formateamos el dataset

Función que hace el nuevo formato de campos del dataset

In [ ]:
def format_instruction(example):
    # Obtenemos la instrucción y el contexto o input
    texts = re.split("### Question:\n|### Context:\n|### Response:\n",example['input'])[1:3]
    # Creamos el prompt en español
    es_prompt = ("Dada la instrucción siguiente que describe una tarea o pregunta, junto con su contexto o entrada correspondiente que proporciona información para resolver la tarea. "
                    "Escribe una respuesta que complete adecuadamente la petición. \n" +
                    "### Instrucción: {} \n" +
                    "### Contexto: {} \n" +
                    "### Respuesta: ")

    prompt_es = es_prompt.format(texts[0].strip(), texts[1].strip())
    #output = [{"instruction": texts[0].strip(), "input": texts[1].strip(), "output": generation.strip()} for generation in example['generations']]
    output = {"instruction": texts[0].strip(), "context": texts[1].strip(), "output": example['generations'][0].strip(), "prompt_es": prompt_es}
    # Creamos el prompt en español


    return output

Comprobamos si algun ejemplo tiene mas de 1 respuesta generada, si es aasi habría que separarlo.

In [ ]:
i=0
for example in dataset['train']:
    if len(example['generations'])>1:
        print(" Example: ",i," con mas de 1 generacion")
    i+=1

In [ ]:
i

11105

Formatemos el prompt y lo separamos en los 3 campos instruction, input, output

In [ ]:
formatted_dataset = dataset.map(format_instruction, batched = False,)

In [ ]:
# Eliminamos las columnas sobrantes
formatted_dataset = formatted_dataset.remove_columns(column_names=['generation_model', 'generation_prompt','raw_generation_responses','generations'])
formatted_dataset = formatted_dataset.rename_columns({"input": "prompt", "context": "input"})
formatted_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'instruction', 'input', 'output', 'prompt_es'],
        num_rows: 11105
    })
})

# Descargamos el dataset de los textos originales para añadir la fuente a cada registro del nuevo dataset

In [ ]:
source_dataset = load_dataset("edumunozsala/legal-refugiados-es", download_mode='force_redownload', ignore_verifications=True)
source_dataset

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2555: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(


Generating train split:   0%|          | 0/5485 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'page', 'text_ini', 'text_fin', 'text'],
        num_rows: 5485
    })
})

Función para generar los campos source, page y inicio y fin de la fuenta

In [ ]:
def format_source(example):
    try:
        index = source_dataset['train']['text'].index(example['input'])
        source_ini= source_dataset['train']['text_ini'][index] if source_dataset['train']['text_ini'][index] is not None else 0
        source_fin= source_dataset['train']['text_fin'][index] if source_dataset['train']['text_fin'][index] is not None else 0
        result = {"source": source_dataset['train']['source'][index], "page": source_dataset['train']['page'][index],
                  "source_ini": source_ini, "source_fin": source_fin}
    except:
        print("###### EJEMPLO NO ENCONTRADO ######\n", example['input'])
        index = -1
        result = {"source": "No disponible", "page": "0", "source_ini": 0, "source_fin": 0}

    return result

pprint(format_source(formatted_dataset['train'][1]))

{'page': '1',
 'source': 'BOE_1995_5542_consolidado_asilo_y_refugiado',
 'source_fin': 1419,
 'source_ini': 0}


Formateamos cada ejemplo para que incluya los datos de la fuente

In [ ]:
final_dataset = formatted_dataset.map(format_source, batched = False,)
final_dataset

Map:   0%|          | 0/11105 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'instruction', 'input', 'output', 'prompt_es', 'source', 'page', 'source_ini', 'source_fin'],
        num_rows: 11105
    })
})

In [ ]:
from dotenv import load_dotenv
import os
# Load the enviroment variables
load_dotenv()

True

Subimos el dataset al HF Hub

In [ ]:
final_dataset.push_to_hub("instruct-legal-refugiados-es", token=os.environ['HF_TOKEN'])

Lo subimos a la organizacion de SomosNLP



In [ ]:

final_dataset.push_to_hub("somosnlp/instruct-legal-refugiados-es", token=os.environ['HF_TOKEN'])

# **3ª Fase: Limpieza, Revisión y división del dataset**

In [ ]:
!pip install datasets
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import transformers
import datasets
import huggingface_hub
import pandas
import  re
import statistics
from transformers import AutoTokenizer
from datasets import load_dataset_builder
from datasets import get_dataset_split_names
from datasets import load_dataset
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
huggingface_hub.list_datasets

<bound method HfApi.list_datasets of <huggingface_hub.hf_api.HfApi object at 0x79b548d0d960>>

In [ ]:
get_dataset_split_names("edumunozsala/instruct-legal-refugiados-es")
dataset_builder = load_dataset_builder("edumunozsala/instruct-legal-refugiados-es")
dataset = load_dataset("edumunozsala/instruct-legal-refugiados-es",split="train")

Generating train split:   0%|          | 0/11105 [00:00<?, ? examples/s]

In [ ]:
dataset[0]["instruction"]
n_rows = len(dataset)
n_columns = len(dataset[0])

# FILTRAR DATASET
Primer paso: Filtrado de ejemplos cuyos outputs hicieran match con la expresión regular: "^if$|#|^|~".

Segundo paso: Filtrado de ejemplos con outputs por debajo de 25 tokens. Los tokens son el resultado de hacer split por espacios en blanco.


In [ ]:
pattern = re.compile("^if$|#|\^|~")
clean_indexes = []
remove_indexes = []
for n,r in enumerate(dataset["output"]):
  tokens = re.split(" ",r)
  control = []
  for t in tokens:
    if re.search(pattern,t):
      control.append("poison")
    else:
      control.append("clean")
  if "poison" in control:
    remove_indexes.append(n)
  else:
    clean_indexes.append(n)
  if (len(tokens)<25) and (n not in remove_indexes):
    remove_indexes.append(n)
    clean_indexes.remove(n)
    #print(n)
    #print(dataset[n]["instruction"])
    #print(dataset[n]["output"])
dataset_filter = dataset.select(clean_indexes)
print(len(dataset_filter))

10326


# GUARDAR FILTRADO EN CSV

In [ ]:
#to_csv('dataset\instruct-legal-es\instruct-legal-chat-es.csv', batch_size=10, header=True)
#dataset_filter.to_csv('/qa_refugiados/dataset_filtered.csv', batch_size=10, header=True)
%pwd
#%cd /content/drive/MyDrive/qa_refugiados
%cd /content/drive/MyDrive
dataset_filter.to_csv('dataset_filtered.csv', batch_size=10, header=True)

/content/drive/MyDrive


Creating CSV from Arrow format:   0%|          | 0/1033 [00:00<?, ?ba/s]

68340563

# CREAR TEST SET
Primer paso: Se calculó la media y desviación estándar del número de tokens tanto en la instructions como en los outputs por separado.

Segundo paso: Se seleccionaron los ejemplos cuyas instruction y outputs estuvieran dentro del límite "media +- 0,35*desviación_estándar".


In [ ]:
list_n_tokens_output = []
for r in dataset_filter["output"]:
  n_tokens = float(len(re.split(" ",r)))
  list_n_tokens_output.append(n_tokens)
mean_output = statistics.fmean(list_n_tokens_output)
stdev_output = statistics.stdev(list_n_tokens_output)
limit_output = mean_output + 0.35 * stdev_output
neg_limit_output = mean_output - 0.35 * stdev_output
print(mean_output)
print(stdev_output)

#count = 0
test_indexes_output = []
for i,n in enumerate(list_n_tokens_output):
  if (n>neg_limit_output) and (n<limit_output):
    #count += 1
    test_indexes_output.append(i)

#print(count)

226.4719155529731
83.32506394480212


In [ ]:
list_n_tokens = []
for r in dataset_filter["instruction"]:
  n_tokens = float(len(re.split(" ",r)))
  list_n_tokens.append(n_tokens)
mean_input = statistics.fmean(list_n_tokens)
stdev_input = statistics.stdev(list_n_tokens)
limit = mean_input + 0.35*stdev_input
neg_limit = mean_input - 0.35*stdev_input
print(mean_input)
print(stdev_input)
#count = 0
test_indexes = []
for i,n in enumerate(list_n_tokens):
  if (n>neg_limit) and (n<limit):
    #count += 1
    test_indexes.append(i)
print(test_indexes)

22.041061398411777
5.680539738082932
[14, 18, 20, 23, 26, 28, 29, 31, 38, 41, 42, 43, 49, 50, 57, 59, 69, 78, 84, 92, 94, 100, 109, 111, 117, 126, 132, 133, 146, 152, 153, 175, 179, 181, 184, 187, 189, 190, 192, 199, 202, 203, 204, 210, 211, 218, 220, 230, 239, 244, 246, 251, 254, 256, 257, 258, 262, 266, 274, 275, 279, 289, 293, 295, 299, 306, 315, 316, 317, 318, 322, 327, 336, 341, 356, 364, 368, 371, 372, 375, 376, 377, 381, 382, 384, 385, 387, 390, 393, 395, 396, 400, 402, 404, 420, 421, 430, 433, 435, 439, 440, 441, 443, 450, 453, 454, 455, 461, 463, 464, 473, 475, 476, 479, 480, 484, 489, 495, 497, 502, 517, 520, 528, 532, 533, 535, 539, 540, 541, 544, 546, 547, 549, 552, 554, 555, 558, 559, 561, 562, 570, 574, 576, 578, 580, 588, 593, 600, 602, 608, 629, 633, 647, 649, 656, 661, 662, 673, 674, 682, 683, 688, 689, 702, 717, 721, 723, 725, 727, 728, 730, 732, 733, 735, 740, 745, 750, 753, 754, 755, 756, 762, 768, 769, 781, 783, 787, 789, 790, 793, 796, 801, 803, 812, 815, 819, 824

In [ ]:
count = 0
both_test_indexes = []
for i in test_indexes:
  if i in test_indexes_output:
    count += 1
    both_test_indexes.append(i)

In [ ]:
dataset_test = dataset_filter.select(both_test_indexes)
print(len(dataset_test))
dataset_test[0]["output"]
dataset_filter[0]["output"]

896


'La Ley 5/1984, relativa al derecho de asilo y a la condición de refugiado, se basa en los siguientes principios fundamentales:\n\n1. Adecuación a la Convención de Ginebra de 1951 y el Protocolo de 1967 sobre el Estatuto de los Refugiados: La ley remite reiteradamente a estos acuerdos internacionales, lo que significa que sus principios y disposiciones son directamente aplicables en España en materia de asilo y refugio.\n\n2. Configuración de un marco procedimental general: La ley establece el marco general para la tramitación de solicitudes de asilo, introduciendo un procedimiento de inadmisión a trámite para evitar el uso fraudulento del sistema de protección a refugiados con fines de inmigración económica.\n\n3. Procedimiento de determinación de la condición de refugiado: El Real Decreto 203/1995, que aprueba el Reglamento de aplicación de la Ley 5/1984, detalla el procedimiento para determinar el reconocimiento de la condición de refugiado, asegurando que se sigan las normas y gara

In [ ]:
#Remove test from filtered train

limit = len(dataset_filter)
train_indexes = [n for n in range(0,limit) if n not in both_test_indexes]
print(train_indexes)
dataset_train = dataset_filter.select(train_indexes)
print(len(dataset_filter))
print(len(dataset_test))
print(len(dataset_train))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 176, 177, 178, 179, 180, 181, 182, 183, 185, 186, 188, 189, 190, 191, 193, 194, 195, 196, 197, 198, 200, 201, 202, 203, 205, 206, 207, 208, 209, 210, 212, 213, 214, 215, 216, 217, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 231, 232, 233, 234, 235, 236, 237, 238, 240, 241, 242, 243,

In [ ]:
dataset_test.to_csv('dataset_test.csv', batch_size=10, header=True)
dataset_train.to_csv('dataset_train_clean.csv', batch_size=10, header=True)

Creating CSV from Arrow format:   0%|          | 0/90 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/943 [00:00<?, ?ba/s]

62144416